<H2>Diffusion Models</H2>

Insert an image here that illustrates the model architecture.<br>
Training Instance X $\rightarrow\rightarrow\rightarrow$ Latent Instance Z<br>
Latent Instance Z $\rightarrow$ Training Instance X<br>
Randomly Generated Latent Instance Z' $\rightarrow$ New Instance X' similarly distributed as X<br>

Diffusion models are a class of latent variable generative models capable of generating samples that are distributed "similarly" as the training dataset. Learning is achieved by applying a series of forward Diffusion models apply progressively more noise to the data during a sequence of transformation steps. A neural network can be trained to learn how to generate samples

A diffusion model learns how do generate samples that are "similar" to the data it trains on by applying a series of forward and reverse diffusion processes. The forward process will be a fixed transformation while the reverse process will be a neural network.

$x_{0}\overset{forward (encoder)}{\underset{reverse (decoder)}\rightleftarrows} x_{1:T}$<br>

<H3>Forward Diffusion Process Using Diagonal Gaussian</H3>

A forward diffusion process of $T\in(0,\inf)$ sequential transformations, $\xrightarrow{\text{forward}}$, to a random variable $x_{0}$ 

$x_{0}\rightarrow x_{1}\rightarrow\dots\rightarrow x_{T}$<br>

Each latent variable $x_{t}\in x_{1:T}$ consists of the sum of noise and the value of $x_{t-1}$.<br>
The joint distribution $q(x_{1:T}|x_{0})$ of latent variables $x_{1:T}$ given the original manifest variable $x_{0}$ is memoryless.<br>
$q(x_{t:T}|x_{0})=\prod\limits_{t=1}^{T}{q(x_{t}|x_{t-1})}\quad\quad\quad\quad\text{(Markov Property)}$<br>

Ho et al. [3] show that a reparameterization as follows provides 

Starting at point $x_{0}\sim\ q$, where $q$ is a probability distribution that will be learned, add noise to the original data in the following fashion:<br>
$x_{t}=\sqrt{1-\beta_{t}}x_{t-1}+\sqrt{\beta_{t}}z_{t}$<br>
$\text{where}$<br>
$z_{1},\dots,z_{T}\text{ are I.I.D. }\mathcal{N}(0,I)$<br> 

 Let $q(x_{t}|x_{t-1})\sim\mathcal{N}(x_{t};\sqrt{1-\beta_{t}}x_{t-1},\beta_{t}I)\quad\quad\quad\quad\quad\quad\quad$<br>
$\text{where:}\quad\quad\quad\beta_{t}\text{is the variance at time t}$<br>
$\quad\quad\quad\quad\quad\quad \beta_{1}\lt\beta_{2}\lt\dots<\beta_{T}$<br>
$\quad\quad\quad\quad\quad\quad \beta_{t}\in(0, 1)$<br>

Axiom: $q(x_{T}|x_{0})\rightarrow\mathcal{N}(0,I)$ as $T\rightarrow\inf$.<br>

Let's constrain ourselves to a versy small $B_{t}\ll1$. There are claims (refs??) that using a small step size makes the reverse process of learning the diffusion process more stable.<br>

<H3>Note on variance of ELBO using this approach</H3>

Axiom: This variational lower bound (aka the Evidence Lower Bound - ELBO) has high variance.<br>
Ho et al. [3] show that, in addition to a reparameterization trick discussed in the following section, 
sampling from any forward step using an additional auxillary noise variable $\epsilon$ reduces variance.<br>
$x_{t}=\sqrt{\bar{\alpha_{t}}}x_{t-1}+\sqrt{1-\bar{\alpha_{t}}}\epsilon$<br>
$\text{where}$<br>
$\epsilon\sim\mathcal{N}(0,I)\text{ and are I.I.D. }$<br> 


<H3>Reverse Diffusion Process Using Diagonal Gaussian Kernel</H3>

Now that we have a fully developed latent space, we can train a neural network to learn the parameters needed to generate similarly distributed samples.

According to Feller's "On the Theory of Stochastic Processes, with Particular Reference to Applications" (ref??), claim states that diffusion models can and will converge to an optimal solution.

$x_{0}\leftarrow\dots\leftarrow x_{T-1}\leftarrow x_{T}$

$p_\theta(x_{0:T})=p(x_{T})\prod\limits_{t=1}^{T}p_{\theta}(x_{t-1}|x_{t})$<br>
where<br>
$p_{\theta}(x_{T})\sim\mathcal{N}(x_{T}; 0, I)$<br>
and<br>
$p_\theta(x_{t-1}|x_{t})\sim\mathcal{N}(x_{t-1};\mu_\theta(x_{t}, t), \Sigma_\theta(x_{t}, t))$<br>


The negative log-likelihood $-p_{\theta}(x_{0}) serves as an appropriate loss function however computing the marginal distribution $p_{\theta}(x_{0})$ requires integration over all 1:T latent dimensions which becomes intractable quickly.
$p_{\theta}(x_{0})=\int p_{\theta}(x_{0:T}dx_{1:T}$<br>

One solution to the dimensionallity problem is to use a variational lower bound.<br>
$log\ p_\theta(x)\ge\text{variational lower bound (ELBO)}$ which is based upon the KL-Divergence $D_{KL}(p||q)=\int\ p(x)\ log\ \frac{p(x)}{q(x)}dx$<br>

$log\ p_\theta(x_{0})\ge\mathbb{E}_{q(x_{1:T}|x_{0})}[log\ p_{\theta}(x_{0}|x_{1:T})] - D_{KL}(q(x_{1:T}|x_{0})\ ||\ p_{\theta}(x_{1:T}))$<br>
$\quad\quad\quad\quad\quad=\mathbb{E}_{q(x_{1:T}|x_{0})}[log\ p_{\theta}(x_{0}|x_{1:T})] - \mathbb{E}_{q(x_{1:T}|x_{0})}\left[\frac{q(x_{1:T}|x_{0})}{p_{\theta}(x_{1:T})}\right]$<br>
$\quad\quad\quad\quad\quad=\mathbb{E}_{q(x_{1:T}|x_{0})}\left[log\ p_{\theta}(x_{0}|x_{1:T})+log\ \frac{p_{\theta}(x_{1:t})}{q(x_{1:T}|x_{0})} \right]$<br>
$\quad\quad\quad\quad\quad=\mathbb{E}_{q(x_{1:T}|x_{0})}\left[log\ p_{\theta}(x_{0})+\sum\limits_{t\ge1}\ log\frac{p_{\theta}(x_{t-1}|x_{t})}{q(x_{t}|x{t-1})}\right]$<br>
$\quad\quad\quad\quad\quad=\mathbb{E}_{q(x_{1:T}|x_{0})}\left[log\ \frac{p_{\theta}(x_{0:T})}{q(x_{1:T}|x_{0})}\right]$<br>
$\quad\quad\quad\quad\quad=\mathbb{E}_{q(x_{1:T}|x_{0})}\left[log\ p_{\theta}(x_{t})+\sum\limits_{t\ge1}log\frac{p_{\theta}(x_{t-1}|x_{t})}{q(x_{t}|x_{t-1})}\right]$<br>

Axiom: Any forward step of the process can be directly sampled as shown here:<br>
$q(x_{t}|x_{0})=\mathcal{N}(x_{t};\sqrt{\bar{\alpha}_{t}}x_{0},(1-\bar{\alpha}_{t})I)$<br>
where<br>
$\bar{\alpha}_{t}=\prod\limits_{s=1}^{t} a_{s}$<br>
and<br>
$\alpha_{s}=1-\beta_{s}$<br>


<H4>Alternative Formulation of the ELBO Objective for Reduced Variance</H4>

Sohl-Dickstein demonstrate the derivation of a reduced objective [1, Appendix B] which is shown below:
$\mathbb{E}_{q_(x_{1:T}|x_{0})}\left[-D_{KL}(q(x_{T}|x_{0})||p_{\theta}(x_{T}))-\sum\limits_{t>1}D_{KL}(q(x_{t-1}|x+{t}, x_{0})\ ||\ p_{\theta}(x_{t-1}|x_{t}))+\ log\ p_{\theta}(x_{0}|x_{1})\right]$<br>


$p_{\theta}(x_{T})$ is fixed since this is just the last(first) step in the forward(reverse) process.<br>
$q(x_{t}|x_{0})$ is also fixed ... (insert rational here)<br>
therefore the first term of the above expectation can be simplified to:<br>
$\mathbb{E}_{q_(x_{1:T}|x_{0})}\left[(0)-\sum\limits_{t>1}D_{KL}(q(x_{t-1}|x+{t}, x_{0})\ ||\ p_{\theta}(x_{t-1}|x_{t}))+\ log\ p_{\theta}(x_{0}|x_{1})\right]-C$<br>

Both of the $q(x_{t-1}|x+{t}, x_{0})$ and $p_{\theta}(x_{t-1}|x_{t}))$ arguments of the KL-Divergence term are both Gaussian and therefore the KL-Divergence term can be evaluated in closed form. This also reduces variance during the training process because instead of aiming to reconstruct Monte Carlo samples, targeting of the reverse step is the true posterior of forward process given $x_{0}$. (!insert ref here)<br>

<H3>Implementation of Reverse Diagonal Normal Diffusion Process</H3>

Ho et al. [3] show that fixing the covariance parameter of the gaussian (shown again for convenience below) to time based constants allows us to learn the parameter for the means $\mu_{\theta}(x_{t},t)$<br>

$p_{\theta}(x_{t-1}|x_{t})\sim\mathcal{N}(x_{t-1};\mu_{\theta}(x+_{t},t),\sum_{\theta}(x_{t},t))$<br>

<H3>Loss Function</H3>


The negative log-likelihood serves as the basis for the loss function that will be used in the backwards pass of the neural network tasked with learning to predict the noise (see "Additional Notes On Ho et al." section below).<br>
$-log(p_{\theta}(x_{0}))$<br>

$loss=\mathbb{E}_{x_{0},\ \epsilon,\ t}\left[w_{t}||\epsilon - \epsilon_{\theta}(x_{t},t)||^{2}\right]$<br>

Ho et al. [3] demonstrate that more realistic sampling can be obtained by eliminating the step-specific weighting term $w_{t}$.<br>
$loss=\mathbb{E}_{x_{0},\ \epsilon,\ t}\left[(1)||\epsilon - \epsilon_{\theta}(x_{t},t)||^{2}\right]$<br>

<H3>Model Architecture U-Net</H3>

Ho et al. Describe the model architecture they used in [3].
1. 

<H3>Training Algorithm</H3>

$\text{1.}\quad\text{repeat}$<br>
$\text{2.}\quad\quad\ x_{0}\sim\ q({x_{0}})$<br>
$\text{3.}\quad\quad\ t\sim\text{Uniform}({1,\dots,T})$<br>
$\text{4.}\quad\quad\ \epsilon\sim\mathcal{N}(0,I)$<br>
$\text{5.}\quad\quad\ \text{Take gradient descent step on}$<br>
$\quad\quad\quad\quad\nabla_{\theta}\text{\\}\epsilon\ -\ \epsilon_{\theta}(\sqrt{\bar{\alpha_{t}}}x_{0} + \sqrt{1-\bar{\alpha_{t}}}\epsilon,t)\text{||}^{2}$<br>
$\text{6.}\quad\ \text{until converged.}$<br>


<H3>Sampling Algorithm</H3>

$\text{1.}\quad\ x_{T}\sim\mathcal{N}(0,I)$<br>
$\text{2.}\quad\quad\ \text{for t=T,...,1 do}$<br>
$\text{3.}\quad\quad\quad\quad\ z\sim\mathcal{N}(0,I)\quad\text{if }t>1,\text{else }z=0$<br>
$\text{4.}\quad\quad\quad\quad\ x_{t-1}=\frac{1}{\sqrt{\alpha_{t}}}\left(x_{t}-\frac{1-\alpha_{t}}{\sqrt{1-\bar{alpha_{t}}}}\epsilon_{\theta}(x_{t},t)\right)\sigma_{t}z$<br>
$\text{5.}\quad\quad\ \text{end for}$<br>
$\text{6.}\quad\quad\ \text{return }x_{0}$<br>


<H3>Additional Notes On Ho et al. [3]</H3>

1. Authors conclude that a Denoising Diffusion Probabilistic Model (DDPM) can predict A.) Mean of the noise, B.) Original Image, and C.) Noise of the image.
2. Authors also conclude that B is intractable.
3. Authors conclude that A and C are reparamaterizations of the same problem.

<H2>References</H2>

1. Sohl-Dickstein J. et al., "Deep Unsupervised Learning using
Nonequilibrium Thermodynamics" arXiv:1503.03585v, rev. Nov. 18, 2015.
2. Feller W., "On the Theory of Stochastic Processes, with Particular Reference to Applications", 
3. Ho, J. et al., "Denoising Diffuision Probabilistic Models", Dec. 16, 2020, arXi:200611239v2. 
